## Import dos dados

In [85]:
import string
from pathlib import Path
from collections import Counter
import math
import re
import pandas as pd
import numpy as np

In [86]:
datasets_dir = Path().absolute().parent.joinpath('datasets')
reviews = f"{datasets_dir}/fakenews.csv"

In [87]:
df = pd.read_csv(reviews)
df.head()

,index,label,preprocessed_news
0,0,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,fake,reinaldo azevedo desmascarado policia federal ...
3,3,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,fake,radialista americano fala sobre pt vendem ilus...


In [88]:
df.groupby('label').count()

,index,preprocessed_news
label,,
TRUE,3600,3600
fake,3600,3600


In [89]:
fake_sentences = 3600
true_sentences = 3600

## Tratamento do texto e treinamento

In [90]:
stopwords = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 
             'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 
             'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas' 
             'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 
             'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 
             'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 
             'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 
             'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 
             'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam', '.', ',']

true_rev = []
fake_rev = []

In [91]:
del df['index']

In [92]:
for index, row in df.iterrows():
    if(row['label'] == 'fake'):
        fake_rev.append(row['preprocessed_news'])
    elif (row['label'] == 'TRUE'):
        true_rev.append(row['preprocessed_news'])

In [93]:
len(true_rev)

3600

In [94]:
#Separando para testes
training_fake_news = fake_rev[:2700]
training_true_news = true_rev[:2700]
training_news = training_fake_news + training_true_news
testing_fake_news = fake_rev[2701:]
testing_true_news = true_rev[2701:]
testing_news = testing_fake_news + testing_true_news

In [95]:
len(testing_news)

1798

In [96]:
def remove_stopword(dictionary):
    #delete = [key for key in dictionary if key in stopwords]
    #for key in delete: del dictionary[key]
    return dictionary     

In [97]:
def format_text(array):
    normalized = [str.lower(sentence) for sentence in array]
    split = [re.findall(r"[\w']+|[,.!?;]", item) for item in normalized]
    flat_split = []
    for lists in split:
        for words in lists:
            flat_split.append(words)
    return flat_split

In [98]:
def format_text_count(array):
    normalized = [str.lower(sentence) for sentence in array]
    split = [re.findall(r"[\w']+|[,.!?;]", item) for item in normalized]
    counter = [Counter(word) for word in split]
    summed = Counter({})
    for word in counter:
            summed += Counter(word)
    return summed   

In [99]:
fake_word_count = format_text_count(training_fake_news)
true_word_count = format_text_count(training_true_news)

In [100]:
fake_words = remove_stopword(fake_word_count)
true_words = remove_stopword(true_word_count)

In [112]:
print(fake_words['de'])

0


In [101]:
def cal_number_of(dictionary):
    numbers = 0
    for value in dictionary.values():
       numbers += value
    return numbers    

In [102]:
numb_of_fake_words = cal_number_of(fake_words)
numb_of_true_words = cal_number_of(true_words)
numb_of_words = numb_of_fake_words + numb_of_true_words

In [103]:
print(numb_of_words, numb_of_fake_words, numb_of_true_words)

1886153 273333 1612820


## Classificação

In [104]:
#log P(C)
prob_of_fake_math = math.log((fake_sentences) / (true_sentences+fake_sentences), 10)
prob_of_true_math = math.log((true_sentences) / (true_sentences+fake_sentences), 10)

In [105]:
#E log(P(Wi|c))
def calc_prob_of_word_in_context(word, context):
    if(context == 'fake'):
        probability = math.log((fake_words[word] + 1) / (numb_of_fake_words + numb_of_words), 10)
    elif(context =='TRUE'):
        probability = math.log((true_words[word] + 1) / (numb_of_true_words + numb_of_words), 10)
    return probability    

In [106]:
def ascertain_context(review):
    prob_of_fake = 0.0
    prob_of_true = 0.0
    prob_of_words_in_fake = 0.0
    prob_of_words_in_true = 0.0
    for words in review:
        prob_of_words_in_fake += (calc_prob_of_word_in_context(words, 'fake'))
        prob_of_words_in_true += (calc_prob_of_word_in_context(words, 'TRUE'))
    
    prob_of_fake = prob_of_fake_math + prob_of_words_in_fake
    prob_of_true = prob_of_true_math + prob_of_words_in_true
    
    if(prob_of_fake > prob_of_true):
        #print(prob_of_fake, prob_of_true)
        return 'fake'
    elif(prob_of_fake < prob_of_true):
        #print(prob_of_fake, prob_of_true)
        return 'TRUE'

In [117]:
class Classification():
    selected_number = input("Selecione um número de 0 a 1798: ")
    print("News escolhida: ", testing_news[int(selected_number)])
    split_selected_text = format_text([testing_news[int(selected_number)]])
    print(ascertain_context(split_selected_text))

Selecione um número de 0 a 1798:  120


News escolhida:  russia entra briga contra estado islamico putin avisa vamos castigar criminosos
TRUE


In [108]:
class Classification():
    selected_number = input("Selecione um número de 0 a 1798: ")
    print("News escolhida: ", training_news[int(selected_number)])
    print(ascertain_context(training_news[int(selected_number)]))

Selecione um número de 0 a 1798:  0


News escolhida:  katia abreu diz vai colocar expulsao moldura nao reclamar senadora katia abreu disse expulsao pmdb resultado acao cupula atual legenda segundo oportunista amanha vou botar moldura dourada expulsao porque maos onde veio atestado boa conduta curriculo pessoas expulsaram nao servem pais servem pais beneficios proprios disse katia abreu ue expulsao algo tao bom curriculo tanta choradeira katia sabemos motivo provavelmente katia nao valor pt partido ja deveria absorvido parece pt gostava katia somente ficasse entrincheirada dentro pmdb rebaixar demais resta katia ficar chorando pitangas todos cantos tempo ate momento pt nao cadastrou katia abreu fileiras situacao patetica agricultura dilma
TRUE


In [78]:
print(training_news[0])

katia abreu diz vai colocar expulsao moldura nao reclamar senadora katia abreu disse expulsao pmdb resultado acao cupula atual legenda segundo oportunista amanha vou botar moldura dourada expulsao porque maos onde veio atestado boa conduta curriculo pessoas expulsaram nao servem pais servem pais beneficios proprios disse katia abreu ue expulsao algo tao bom curriculo tanta choradeira katia sabemos motivo provavelmente katia nao valor pt partido ja deveria absorvido parece pt gostava katia somente ficasse entrincheirada dentro pmdb rebaixar demais resta katia ficar chorando pitangas todos cantos tempo ate momento pt nao cadastrou katia abreu fileiras situacao patetica agricultura dilma


In [50]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))

ValueError: could not convert string to float: 'katia abreu diz vai colocar expulsao moldura nao reclamar senadora katia abreu disse expulsao pmdb resultado acao cupula atual legenda segundo oportunista amanha vou botar moldura dourada expulsao porque maos onde veio atestado boa conduta curriculo pessoas expulsaram nao servem pais servem pais beneficios proprios disse katia abreu ue expulsao algo tao bom curriculo tanta choradeira katia sabemos motivo provavelmente katia nao valor pt partido ja deveria absorvido parece pt gostava katia somente ficasse entrincheirada dentro pmdb rebaixar demais resta katia ficar chorando pitangas todos cantos tempo ate momento pt nao cadastrou katia abreu fileiras situacao patetica agricultura dilma'